In [5]:
# %%capture
!pip install --no-index /kaggle/input/bitsandbytes0-42-0/bitsandbytes-0.42.0-py3-none-any.whl --find-links=/kaggle/input/bitsandbytes0-42-0
# !pip install --no-index  /kaggle/input/bitsandbytes0-42-0/optimum-1.21.2-py3-none-any.whl --find-links=/kaggle/input/bitsandbytes0-42-0
# !pip install --no-index  /kaggle/input/bitsandbytes0-42-0/auto_gptq-0.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --find-links=/kaggle/input/bitsandbytes0-42-0

Looking in links: /kaggle/input/bitsandbytes0-42-0
Processing /kaggle/input/bitsandbytes0-42-0/bitsandbytes-0.42.0-py3-none-any.whl


In [6]:
%%writefile merge.py
import torch
import gc
lora_dir = '/kaggle/input/ds5-14b-recall-19/epoch_19_model/adapter.bin'
d1 = torch.load(lora_dir,map_location='cpu')
lora_dir = '/kaggle/input/ds5-14b-recall-19-f1/epoch_19_model/adapter.bin'
d2 = torch.load(lora_dir,map_location='cpu')
lora_dir = '/kaggle/input/ds5-14b-recall-19-f2/epoch_19_model/adapter.bin'
d3 = torch.load(lora_dir,map_location='cpu')
lora_dir = '/kaggle/input/ds5-14b-recall-19-f3/epoch_19_model/adapter.bin'
d4 = torch.load(lora_dir,map_location='cpu')
lora_dir = '/kaggle/input/ds5-14b-recall-19-f4/epoch_19_model/adapter.bin'
d5 = torch.load(lora_dir,map_location='cpu')
d = {}

for k, v in d1.items():
    v = d1[k] + d2[k] + d3[k]+ d4[k] + d5[k]
    v = v / 5.
    d[k] = v
torch.save(d,"./final_adapter.bin")
del d,d1,d2,d3,d4,d5
gc.collect()

Writing merge.py


In [7]:
!python merge.py

/kaggle/working/merge.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  d1 = torch.load(lora_dir,map_location='cpu')
/kaggle/working/merge.py:6: FutureWarning: You are usi

In [8]:
%%writefile merge2.py
import torch
import gc
lora_dir = '/kaggle/input/recall-14b-all-dis-72b/adapter.bin'
d1 = torch.load(lora_dir,map_location='cpu')
lora_dir = '/kaggle/input/recall-14b-all-dis-72b-math/adapter.bin'
d2 = torch.load(lora_dir,map_location='cpu')
# lora_dir = '/kaggle/input/ds5-14b-recall-19-f2/epoch_19_model/adapter.bin'
# d3 = torch.load(lora_dir,map_location='cpu')
# lora_dir = '/kaggle/input/ds5-14b-recall-19-f3/epoch_19_model/adapter.bin'
# d4 = torch.load(lora_dir,map_location='cpu')
# lora_dir = '/kaggle/input/ds5-14b-recall-19-f4/epoch_19_model/adapter.bin'
# d5 = torch.load(lora_dir,map_location='cpu')
d = {}

for k, v in d1.items():
    v = d1[k] + d2[k]
    v = v / 2.
    d[k] = v
torch.save(d,"./final_adapter_14b_all.bin")
# del d,d1,d2,d3,d4,d5
# gc.collect()

Writing merge2.py


In [9]:
!python merge2.py

/kaggle/working/merge2.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  d1 = torch.load(lora_dir,map_location='cpu')
/kaggle/working/merge2.py:6: FutureWarning: You are u

In [10]:
%%writefile recall.py
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import gc
import pandas as pd
import pickle
import sys
import numpy as np
from tqdm.autonotebook import trange
from sklearn.model_selection import GroupKFold
import json
import torch
from numpy.linalg import norm
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel,BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
)
import sys
import json
import copy
import warnings
warnings.filterwarnings('ignore')


def apk(actual, predicted, k=25):
    """
    Computes the average precision at k.
    
    This function computes the average prescision at k between two lists of
    items.
    
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=25):
    """
    Computes the mean average precision at k.
    
    This function computes the mean average prescision at k between two lists
    of lists of items.
    
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

def batch_to_device(batch, target_device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

def last_token_pool(last_hidden_states: Tensor,
                    attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

@torch.no_grad()
def inference(df, model, tokenizer, device):
    batch_size = 8
    max_length = 512
    sentences = list(df['query_text'].values)
    pids = list(df['order_index'].values)
    all_embeddings = []
    length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
    sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
    for start_index in trange(0, len(sentences), batch_size, desc="Batches", disable=False):
        sentences_batch = sentences_sorted[start_index: start_index + batch_size]
        features = tokenizer(sentences_batch, max_length=max_length, padding=True, truncation=True,
                             return_tensors="pt")
        features = batch_to_device(features, device)
        with torch.no_grad():
            outputs = model.model(**features)
            embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy().tolist()
        all_embeddings.extend(embeddings)

    all_embeddings = [np.array(all_embeddings[idx]).reshape(1, -1) for idx in np.argsort(length_sorted_idx)]

    sentence_embeddings = np.concatenate(all_embeddings, axis=0)
    result = {pids[i]: em for i, em in enumerate(sentence_embeddings)}
    return result

path_prefix = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
# model_path = "/kaggle/input/qwen2.5_14b/transformers/default/1"
# model_path = "/kaggle/input/qwen2.5_14b/transformers/default/1"
model_path = sys.argv[3]
lora_path = sys.argv[1]
save_path = sys.argv[2]
emedding_path = sys.argv[4]
print(lora_path)
print(save_path)
# lora_path="/kaggle/input/ds5-14b-recall-19/epoch_19_model/adapter.bin"
device='cuda:0'
VALID = False


device = 'cuda:0'
tokenizer = AutoTokenizer.from_pretrained(model_path)
bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
model = AutoModel.from_pretrained(model_path, quantization_config=bnb_config,device_map=device)
config = LoraConfig(
        r=64,
        lora_alpha=128,
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],
        bias="none",
        lora_dropout=0.05,  # Conventional
        task_type="CAUSAL_LM",
    )
model = get_peft_model(model, config)
d = torch.load(lora_path, map_location=model.device)
model.load_state_dict(d, strict=False)
model = model.eval()
model_0 = model.to(device)

device = "cuda:1"
model = AutoModel.from_pretrained(model_path, quantization_config=bnb_config,device_map=device)
config = LoraConfig(
        r=64,
        lora_alpha=128,
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],
        bias="none",
        lora_dropout=0.05,  # Conventional
        task_type="CAUSAL_LM",
    )
model = get_peft_model(model, config)
d = torch.load(lora_path, map_location=model.device)
model.load_state_dict(d, strict=False)
model = model.eval()
model_1 = model.to(device)


task_description = 'Given a math question and a misconcepte incorrect answer, please retrieve the most accurate reason for the misconception.'
if VALID:
    tra = pd.read_parquet("/kaggle/input/v1-parquet/v1_val.parquet")
    print(tra.shape)
else:
    tra = pd.read_csv(f"{path_prefix}/test.csv")
    print(tra.shape)
misconception_mapping = pd.read_csv(f"{path_prefix}/misconception_mapping.csv")
if tra.shape[0]<10:
    misconception_mapping = misconception_mapping.sample(n=25,random_state=2023)
    
    
if VALID:
    train_data = []
    for _,row in tra.iterrows():
        for c in ['A','B','C','D']:
            if str(row[f"Misconception{c}Id"])!="nan":
                # print(row[f"Misconception{c}Id"])
                real_answer_id = row['CorrectAnswer']
                real_text = row[f'Answer{real_answer_id}Text']
                query_text =f"###question###:{row['SubjectName']}-{row['ConstructName']}-{row['QuestionText']}\n###Correct Answer###:{real_text}\n###Misconcepte Incorrect answer###:{row[f'Answer{c}Text']}"
                row['query_text'] = get_detailed_instruct(task_description,query_text)
                row['answer_id'] = row[f"Misconception{c}Id"]
                train_data.append(copy.deepcopy(row))
    train_df = pd.DataFrame(train_data)
    train_df['order_index'] = list(range(len(train_df)))
else:
    train_data = []
    for _,row in tra.iterrows():
        for c in ['A','B','C','D']:
            if c ==row['CorrectAnswer']:
                continue
            if f'Answer{c}Text' not in row:
                continue
            real_answer_id = row['CorrectAnswer']
            real_text = row[f'Answer{real_answer_id}Text']
            query_text =f"###question###:{row['SubjectName']}-{row['ConstructName']}-{row['QuestionText']}\n###Correct Answer###:{real_text}\n###Misconcepte Incorrect answer###:{row[f'Answer{c}Text']}"
            query_text2 = f"###question###:{row['SubjectName']}-{row['ConstructName']}-{row['QuestionText']}\n###Correct Answer###:{real_text}\n###Incorrect distractor answer###:{row[f'Answer{c}Text']}"
            row['query_text'] = get_detailed_instruct(task_description,query_text)
            row['query_text2'] = query_text2
            row['answer_name'] = c
            train_data.append(copy.deepcopy(row))
    train_df = pd.DataFrame(train_data)
    train_df['order_index'] = list(range(len(train_df)))
train_df.shape


temp1 = train_df.iloc[0::2].copy()
temp2 = train_df.iloc[1::2].copy()
with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (temp1, temp2), (model_0, model_1), (tokenizer, tokenizer),('cuda:0','cuda:1'))
train_embeddings={}
for res in list(results):
    train_embeddings.update(res)
# train_embeddings = inference(train_df, model, tokenizer, device)


# misconception_mapping['query_text'] = misconception_mapping['MisconceptionName']
# misconception_mapping['order_index'] = misconception_mapping['MisconceptionId']


# temp1 = misconception_mapping.iloc[0::2].copy()
# temp2 = misconception_mapping.iloc[1::2].copy()
# with ThreadPoolExecutor(max_workers=2) as executor:
#     results = executor.map(inference, (temp1, temp2), (model_0, model_1), (tokenizer, tokenizer),('cuda:0','cuda:1'))
# # doc_embeddings = results[0].undate(results[1])

# doc_embeddings={}
# for res in list(results):
#     doc_embeddings.update(res)
# doc_embeddings = inference(misconception_mapping, model, tokenizer, device)

# sentence_embeddings = np.concatenate([e.reshape(1, -1) for e in list(doc_embeddings.values())])
# index_text_embeddings_index = {index: paper_id for index, paper_id in
#                                          enumerate(list(doc_embeddings.keys()))}


with open(emedding_path,'rb') as f:
    index_text_embeddings_index,sentence_embeddings = pickle.load(f)

predicts_test = []
predicts_probs = []
for _, row in tqdm(train_df.iterrows()):
    query_id = row['order_index']
    query_em = train_embeddings[query_id].reshape(1, -1)
    
    # 计算点积
    cosine_similarity = np.dot(query_em, sentence_embeddings.T).flatten()
    
    # 对余弦相似度进行排序并获取前100个索引
    sort_index = np.argsort(-cosine_similarity)
    pids = [index_text_embeddings_index[index] for index in sort_index]
    predicts_test.append(pids)
    predicts_probs.append([cosine_similarity[index] for index in sort_index])
    

if VALID:
    train_df['recall_ids'] = predicts_test
    print(mapk([[data] for data in train_df['answer_id'].values],train_df['recall_ids'].values))
else:
    train_df['recall_ids'] = predicts_test
    train_df['recall_probs'] = predicts_probs
    train_df['MisconceptionId'] = [' '.join(map(str,c[:25])) for c in predicts_test]
    sub = []
    for _,row in train_df.iterrows():
        sub.append(
            {
                "QuestionId_Answer":f"{row['QuestionId']}_{row['answer_name']}",
                "MisconceptionId":row['MisconceptionId']
            }
        )
    submission_df = pd.DataFrame(sub)
    submission_df.to_csv(save_path, index=False)
    train_df.to_parquet(f"./{save_path}.parquet",index=False)
    print("Submission file created successfully!")

Writing recall.py


In [11]:
!python recall.py ./final_adapter.bin  submission1.csv /kaggle/input/qwen2.5_14b/transformers/default/1 /kaggle/input/final-eedi-index-text-embeddings-index/final_eedi_index_text_embeddings_index.pkl 

./final_adapter.bin
submission1.csv
Loading checkpoint shards: 100%|██████████████████| 8/8 [02:08<00:00, 16.10s/it]
(3, 11)
Batches: 100%|████████████████████████████████████| 1/1 [00:14<00:00, 14.73s/it]
9it [00:00, 106.44it/s]
Submission file created successfully!


In [12]:
!python recall.py /kaggle/input/14b-recall-dis-coder-merge/14b_recall_dis_coder_merge.bin  submission2.csv /kaggle/input/qwen2.5-coder/transformers/14b-instruct/1 /kaggle/input/final-eedi-index-text-embeddings-index-coder/final_eedi_index_text_embeddings_index_coder.pkl

/kaggle/input/14b-recall-dis-coder-merge/14b_recall_dis_coder_merge.bin
submission2.csv
Loading checkpoint shards: 100%|██████████████████| 6/6 [02:04<00:00, 20.83s/it]
(3, 11)
Batches: 100%|████████████████████████████████████| 1/1 [00:16<00:00, 16.79s/it]
9it [00:00, 113.60it/s]
Submission file created successfully!


In [13]:
!python recall.py ./final_adapter_14b_all.bin  submission3.csv /kaggle/input/qwen2.5_14b/transformers/default/1 /kaggle/input/recall-14b-dis-70b-all-em/final_eedi_index_text_embeddings_index_14b_all.pkl 

./final_adapter_14b_all.bin
submission3.csv
Loading checkpoint shards: 100%|██████████████████| 8/8 [02:07<00:00, 15.89s/it]
(3, 11)
Batches: 100%|████████████████████████████████████| 1/1 [00:17<00:00, 17.12s/it]
9it [00:00, 111.35it/s]
Submission file created successfully!


In [14]:
import pandas as pd
import numpy as np
val = pd.read_parquet("submission1.csv.parquet")
val2 = pd.read_parquet("submission2.csv.parquet")
val3 = pd.read_parquet("submission3.csv.parquet")
val['recall_probs2'] = val2['recall_probs'].values
val['recall_ids2'] = val2['recall_ids'].values
val['recall_probs3'] = val3['recall_probs'].values
val['recall_ids3'] = val3['recall_ids'].values
res = []
for _,row in val.iterrows():
    score1 =  row['recall_probs']
    score2 = row['recall_probs2']
    score3 = row['recall_probs3']
    score1 = (score1 - score1.min()) / (score1.max() - score1.min())
    score2 = (score2 - score2.min()) / (score2.max() - score2.min())
    score3 = (score3 - score3.min()) / (score3.max() - score3.min())
    mp = {}
    for i in range(len(score2)):
        mp[row['recall_ids'][i]] = mp.get(row['recall_ids'][i],0) + score1[i]
        mp[row['recall_ids2'][i]] = mp.get(row['recall_ids2'][i],0) + score2[i]
        mp[row['recall_ids3'][i]] = mp.get(row['recall_ids3'][i],0) + score3[i]
    mp_ = sorted(mp.items(),key=lambda x: -x[1])[:25]
    res.append([_[0] for _ in mp_])
val['recall_ids'] = res
sub=[]
for _,row in val.iterrows():
        sub.append(
                {
                    "QuestionId_Answer":f"{row['QuestionId']}_{row['answer_name']}",
                    "MisconceptionId":' '.join(map(str,row['recall_ids'][:25]))
                }
        )
sub = pd.DataFrame(sub)
print(sub.head(5))
sub.to_csv(f"submission.csv", index=False)
# val.to_parquet("rank.parquet")

  QuestionId_Answer                                    MisconceptionId
0            1869_B  2306 1507 706 2181 1005 2532 1672 1516 2518 13...
1            1869_C  2306 1507 1005 706 2532 2518 1963 1345 1054 21...
2            1869_D  1672 328 1507 2532 1005 1516 706 2306 1941 218...
3            1870_A  2142 2068 418 1755 1535 143 167 891 1904 2372 ...
4            1870_B  143 891 2398 2078 2142 2068 1755 363 167 80 15...
